## 什么是 Artifacts
`artifacts` 是自 0.2.19 版本之后引入的一个全新概念，是构建 LLM 应用（尤其是复杂 Agent 系统）中的关键组成部分。LangChain 开始逐步模块化，提出了“LLM 过程可观测性”的设计理念，不再只追求最终输出，还要记录中间思考路径。一句话解释：
> `artifacts` 是 LLM 在思考过程中生成的中间产物，可以是**文本、图像、文件、代码片段、Prompt**等。它们能被记录、展示、存档，也可以供后续步骤参考或复用。

传统的 LLM 调用只包括三方面：输入 → 模型 → 输出（一次性文本）。LangChain 设计了 `artifacts` 机制，用来：
* 记录所有重要中间产物
* 让 UI / 日志系统展示这些过程
* 让开发者访问 LLM 思考路径中的关键节点


In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,  # 让我们的 Bob 机器人有一些创造性
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

In [12]:
# Artifacts 作为 tool 处理的中间件，当然是从 tool 中获取的
# 下面是个简单的例子，我们手动感受一下 Contents 和 Artifacts 的区别
from typing import Tuple
from langchain_core.tools import tool

# 这是一个简单的工具，LLM 会自动分析出需要传入的 id，然后在数据库里面进行查找
# 假设我们省略其中的 ORM 细节，直接获取到了一条 data
@tool(response_format="content_and_artifact")
def get_person_sql_data(id: int) -> Tuple[str, dict]:
    """We mimic the process of searching someone in the database."""
    ### ...SUPPOSE HERE ARE SOME OPERATIONS OF ORM... ###
    ### ...AND WE FIND THE PERSON... ###
    content = f"Successfully get data from database by 'SELECT * FROM DB WHERE Id={id}'."
    data = {
        "id": "10086",
        "name" : "Nethan Hu",
        "birthday" : "2025-01-01",
        "desc": "active and brilliant"
    }
    return content, data

In [13]:
# 经过测试，我们模拟一下通过 id 来查找“数据”
get_person_sql_data.invoke({"id": 10086})

"Successfully get data from database by 'SELECT * FROM DB WHERE Id=10086'."

但是，可以看到我们并没有获取到SQL中的“数据”，因为不能只用简单的 invoke。

In [10]:
get_person_sql_data.invoke(
    {
        "name": "generate_random_ints",
        "args": {"id": 10086},
        "id": "123456",  # required
        "type": "tool_call",  # required
    }
)

ToolMessage(content="Successfully get data from database by 'SELECT * FROM DB WHERE Id=10086'.", name='get_person_sql_data', tool_call_id='123456', artifact={'id': '10086', 'name': 'Nethan Hu', 'birthday': '2025-01-01', 'desc': 'active and brilliant'})

### 🤔 我们这样做的意义是什么呢？
有时候我们需要 LLM 帮我们查找数据、生成图片、创作文本等，这些数据量通常很大，并且我们希望手动对它们进行再次加工（符合 artifacts 的思想），不想让它再传回到模型本身当中去（况且 LLM 处理这些大量的数据不一定正确）。

比如说刚刚的模型帮我们找到了 id=10086 的这条数据。我们可以在后端对这条数据进行加工处理。在 LangSmith 上我们也可以进行验证，我们发现 LangSmith 只记录了我们的 LLM 的 output 是
```text
Successfully get data from database by 'SELECT * FROM DB WHERE Id=10086'.
```
但是并不会在 LLM 处保存 JSON 信息，只有本地能获取
```json
artifact={'id': '10086', 'name': 'Nethan Hu', 'birthday': '2025-01-01', 'desc': 'active and brilliant'}
```
因此，**只要 LLM 应用有中间步骤/结果、需要展示/存档/追踪，artifact 就是最优解。** 尤其适合写作链、图像链、RAG、代码生成、数据分析等场景。